In [1]:
library(reshape2)
library(dplyr)
meta <- read.csv( paste0("/data/shanlan/crc_trio/1.sub_analysis/plot/","TableS1_sgWGS_mutMeta.csv"), header = T, sep = ',' ) %>% dplyr::select(c(1,18:33))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
#incomplete morphology
#library artifacts
#N_biased: 'FAP03_N2_R4_G6
#NameError(VarifyID): 'FAP03_P1_R1_G3','FAP03_P1_R2_G12','FAP03_P3_R2_G1','FAP03_P5_R1_G5','FAP03_P5_R2_G10'
#'FAP01_P6_R2_G6','FAP01_P4_R5_G5': Manual Review by mutational Signatures 
#discard patients

black_1 = c(
            'FAP03_N2_R4_G6',
            'FAP03_P1_R1_G3','FAP03_P1_R2_G12','FAP03_P3_R2_G1','FAP03_P5_R1_G5','FAP03_P5_R2_G10',
            'FAP01_P6_R2_G6','FAP01_P4_R5_G5'
           )

# lowdepth
black_2 = meta %>% filter(round(depth)<5) %>% dplyr::select(Tumor_Sample_Barcode) %>% t() %>% as.vector()

#low TMB
black_3 = meta %>% filter(TMB > 1000 & round(depth)<7 ) %>% dplyr::select(Tumor_Sample_Barcode) %>% t() %>% as.vector()

#Abnormally high C>A mutation ratio, including Unusual signature distribution by Manual Review
black_4 = meta %>% filter(C.A > 40 | C.T < 20) %>% dplyr::select(Tumor_Sample_Barcode) %>% t() %>% as.vector()

#Presence of multi-hits within numerous same drivers

black_5 = meta %>% filter( driverMut > 50 ) %>% dplyr::select(Tumor_Sample_Barcode) %>% t() %>% as.vector()

# rescued: FAP01_P6_R5_G11
black_list <- setdiff(Reduce(union, list(black_1, black_2, black_3,black_4,black_5)),c('FAP01_P6_R5_G11'))


In [6]:
black_final<-read.table("/data/shanlan/crc_trio/1.sub_analysis/sgWGS_black_list.txt", sep = "\t", header = F)$V1

In [7]:
setdiff(black_list, black_final) 
setdiff(black_final, black_list) 

character(0)

character(0)

In [8]:
cc_list <- '/data/shanlan/crc_trio/1.sub_analysis/runtime/2.sgWGS/tree/sgWGS_ToCC_list0518' %>% read.table() %>% filter(grepl('FAP',V1)) %>% 
dplyr::select(V1) %>% t() %>% as.vector()

In [9]:
dat = meta %>% filter(!Tumor_Sample_Barcode %in% black_list)

In [ ]:
write.table(meta %>% filter(Tumor_Sample_Barcode %in% cc_list) ,"/data/shanlan/crc_trio/1.sub_analysis/sgWGS_meta_toCC.txt", quote = F, sep = "\t", col.names = T, row.names = F)